# Installation

With the help of docker we downloaded a neo4j image locally, and we started the container with the following command 
```
docker run  --platform linux/amd64 -p7474:7474  -p7687:7687 -d --env  NEO4J_AUTH=neo4j/test neo4j:latest
```


# DB Connection 

We will use Python to connect to our database. 
For this reason we should install the neo4j package from PyPI

In [5]:
! pip3 install neo4j

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [6]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "test"), encrypted=False)

# Data Loading

First, we need to read our data from the file system, where they are save as .tsv files. The data were downloaded from
this url: https://snap.stanford.edu/data/act-mooc.html
We will use pandas in order to read our data

In [7]:

import pandas as pd

#Read data from .tsv files
actions = pd.read_csv('act-mooc/mooc_actions.tsv', sep='\t')
labels = pd.read_csv('act-mooc/mooc_action_labels.tsv', sep='\t')
features = pd.read_csv('act-mooc/mooc_action_features.tsv', sep='\t')